In [ ]:
# prompt: data from drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install tensorflow tensorflow-gpu opencv-python matplotlib
!pip install tensorflow

In [ ]:
!pip list


Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.5
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.15.0
backcall                         0.2.0
beautifulsoup4                   4.12.3
bidict                           0.23.1

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from collections import Counter
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Define constants
IMG_HEIGHT, IMG_WIDTH = 150, 150
BATCH_SIZE = 32
NUM_CLASSES = 3

In [ ]:
train_dir = r'/content/drive/MyDrive/Newdataset/Newdataset'
val_dir = r'/content/drive/MyDrive/Newdataset/Newdataset'
test_dir = r'/content/drive/MyDrive/Newdataset/Newdataset'

In [ ]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
val_dataset = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 3000 images belonging to 3 classes.


In [ ]:
test_dataset = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 3000 images belonging to 3 classes.


In [ ]:
# Custom callback to count augmented samples
class SampleCounter(tf.keras.callbacks.Callback):
    def __init__(self):
        super(SampleCounter, self).__init__()
        self.train_counter = Counter()
        self.val_counter = Counter()

    def on_epoch_end(self, epoch, logs=None):
        num_train_batches = len(self.model.train_dataset)
        num_val_batches = len(self.model.val_dataset)

        for i in range(num_train_batches):
            _, y = self.model.train_dataset[i]
            self.train_counter.update(y.argmax(axis=-1))

        for i in range(num_val_batches):
            _, y = self.model.val_dataset[i]
            self.val_counter.update(y.argmax(axis=-1))

        print(f"Epoch {epoch + 1}:")
        print("Training dataset class distribution:", self.train_counter)
        print("Validation dataset class distribution:", self.val_counter)

In [ ]:
# Define the CNN model architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dense(NUM_CLASSES, activation='softmax')  # 3 output classes: mask correctly worn, mask incorrectly worn, no mask
])

In [ ]:
train_dataset = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 3000 images belonging to 3 classes.


In [ ]:
import os
from collections import defaultdict

In [ ]:
def get_class_file_info(directory):
    class_file_info = defaultdict(list)
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            for file_name in os.listdir(class_path):
                file_path = os.path.join(class_path, file_name)
                if os.path.isfile(file_path):
                    class_file_info[class_name].append(file_name)
    return class_file_info

In [ ]:
# Get class file info
train_class_file_info = get_class_file_info(train_dir)
val_class_file_info = get_class_file_info(val_dir)
test_class_file_info = get_class_file_info(test_dir)

In [ ]:
# Print class file info
def print_class_file_info(dataset_name, class_file_info):
    print(f"{dataset_name} dataset class distribution:")
    for class_name, file_names in class_file_info.items():
        print(f"Class '{class_name}' has {len(file_names)} images. Sample files: {file_names[:5]}")  # Print only first 5 file names for brevity

print_class_file_info("Training", train_class_file_info)
print_class_file_info("Validation", val_class_file_info)
print_class_file_info("Test", test_class_file_info)

Training dataset class distribution:
Class 'Incorrect_mask' has 1000 images. Sample files: ['1000.jpg', '1001.jpg', '1004.jpg', '1002.jpg', '1003.jpg']
Class 'With_mask' has 1000 images. Sample files: ['simple107.jpg', 'simple121.jpg', 'simple102.jpg', 'simple122.jpg', 'simple125.jpg']
Class 'Without_mask' has 1000 images. Sample files: ['simple107.jpg', 'simple20.jpg', 'simple115.jpg', 'simple143.jpg', 'simple13.jpg']
Validation dataset class distribution:
Class 'Incorrect_mask' has 1000 images. Sample files: ['1000.jpg', '1001.jpg', '1004.jpg', '1002.jpg', '1003.jpg']
Class 'With_mask' has 1000 images. Sample files: ['simple107.jpg', 'simple121.jpg', 'simple102.jpg', 'simple122.jpg', 'simple125.jpg']
Class 'Without_mask' has 1000 images. Sample files: ['simple107.jpg', 'simple20.jpg', 'simple115.jpg', 'simple143.jpg', 'simple13.jpg']
Test dataset class distribution:
Class 'Incorrect_mask' has 1000 images. Sample files: ['1000.jpg', '1001.jpg', '1004.jpg', '1002.jpg', '1003.jpg']
Clas

In [ ]:
# Print dataset sizes
print(f'Training samples: {train_dataset.n}')
print(f'Validation samples: {val_dataset.n}')
print(f'Testing samples: {test_dataset.n}')

Training samples: 3000
Validation samples: 3000
Testing samples: 3000


In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Attach datasets to the model for the callback
model.train_dataset = train_dataset
model.val_dataset = val_dataset


In [ ]:
 # Train the model
 history = model.fit(
     train_dataset,
     epochs=15,  # Adjust as needed
     validation_data=val_dataset,
     callbacks=[SampleCounter()]
 )

Epoch 1/15
94/94 [==============================] - ETA: 0s - loss: 0.7276 - accuracy: 0.6563Epoch 1:
Training dataset class distribution: Counter({0: 1000, 1: 1000, 2: 1000})
Validation dataset class distribution: Counter({2: 1000, 0: 1000, 1: 1000})
94/94 [==============================] - 819s 9s/step - loss: 0.7276 - accuracy: 0.6563 - val_loss: 0.3495 - val_accuracy: 0.8790
Epoch 2/15
94/94 [==============================] - ETA: 0s - loss: 0.4393 - accuracy: 0.8257Epoch 2:
Training dataset class distribution: Counter({0: 2000, 1: 2000, 2: 2000})
Validation dataset class distribution: Counter({2: 2000, 0: 2000, 1: 2000})
94/94 [==============================] - 372s 4s/step - loss: 0.4393 - accuracy: 0.8257 - val_loss: 0.2702 - val_accuracy: 0.8973
Epoch 3/15
94/94 [==============================] - ETA: 0s - loss: 0.3813 - accuracy: 0.8573Epoch 3:
Training dataset class distribution: Counter({0: 3000, 1: 3000, 2: 3000})
Validation dataset class distribution: Counter({2: 3000, 0: 

In [ ]:
 test_loss, test_acc = model.evaluate(test_dataset)
 print('Test accuracy:', test_acc)


94/94 [==============================] - 73s 775ms/step - loss: 0.1362 - accuracy: 0.9570
Test accuracy: 0.9570000171661377


In [ ]:
test_augmented = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False  # Important to keep the order
)

Found 3000 images belonging to 3 classes.


In [ ]:
# Save the model
model.save('fady_mask_classifier.keras')

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

In [ ]:
# Load the saved model
loaded_model = tf.keras.models.load_model('fady_mask_classifier.keras')

/content/drive/MyDrive/AbdoFata/WithoutMask/100.jpg
/content/drive/MyDrive/AbdoFata/WithoutMask/1000.jpg

/content/drive/MyDrive/AbdoFata/Withmask/1.jpg
/content/drive/MyDrive/AbdoFata/Withmask/1004.jpg


/content/drive/MyDrive/AbdoFata/Incorrect/0.jpg

In [ ]:
# Load and preprocess a single image for testing
img_path = r'/content/drive/MyDrive/AbdoFata/WithoutMask/100.jpg'
img = image.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.0  # Normalize pixel values
prediction = loaded_model.predict(img_array)
# Get class labels
class_labels = list(test_dataset.class_indices.keys())
# Convert prediction to class label
predicted_label = class_labels[np.argmax(prediction)]

1/1 [==============================] - 0s 235ms/step


In [ ]:
print("Predicted Label:", predicted_label)

Predicted Label: Without_mask


In [ ]:
import tensorflow as tf

# Load the existing model
model = tf.keras.models.load_model('fady_mask_classifier.keras')

# Convert the model to the TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open('model_unquant.tflite', 'wb') as f:
    f.write(tflite_model)